## Car price detector

In [2]:
# In this notebook, I will try to predict the price of a car depending on several features. This is a multilinear regression problem.  


In [3]:
# Installation of Pycaret
!pip install pycaret

     |████████████████████████████████| 256kB 3.4MB/s 
     |████████████████████████████████| 174kB 11.0MB/s 
     |████████████████████████████████| 1.8MB 15.2MB/s 
     |████████████████████████████████| 102kB 10.2MB/s 
     |████████████████████████████████| 13.9MB 199kB/s 
     |████████████████████████████████| 266kB 51.7MB/s 
     |████████████████████████████████| 66.1MB 44kB/s 
     |████████████████████████████████| 6.8MB 54.8MB/s 
     |████████████████████████████████| 1.6MB 56.4MB/s 
     |████████████████████████████████| 266kB 49.4MB/s 
     |████████████████████████████████| 2.1MB 48.7MB/s 
     |████████████████████████████████| 51kB 8.6MB/s 
     |████████████████████████████████| 327kB 46.5MB/s 
     |████████████████████████████████| 81kB 12.0MB/s 
     |████████████████████████████████| 153kB 55.7MB/s 
     |████████████████████████████████| 6.0MB 47.3MB/s 
     |████████████████████████████████| 1.1MB 45.9MB/s 
     |████████████████████████████████| 163kB 49.6MB/

In [4]:
# Import the regression package from pycaret
from pycaret.regression import *

In [5]:
# Load the dataset
from google.colab import files
file = files.upload()

Saving CarPrice_Assignment.csv to CarPrice_Assignment.csv


In [6]:
# Import the pandas library to read the dataset
import pandas as pd 
dataset = pd.read_csv('CarPrice_Assignment.csv')
dataset.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,enginetype,cylindernumber,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [ ]:
# The ID column is useless, ltet's drop it
dataset.drop('car_ID', axis=1, inplace=True)

In [ ]:
# I'm going to split the dataset into 2 parts. Data will be 90% of the original one and that I'm going to use for the model's training
# Data_unseen will be the rest on which I'll test the final model on. These observations will be  completely new to the model

data = dataset.sample(frac=0.9, random_state=786)
data_unseen = dataset.drop(data.index)

data.reset_index(drop=True, inplace=True)
data_unseen.reset_index(drop=True, inplace=True)

# Let's see the shape of each dataset (rows, columns)
print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))



Data for Modeling: (184, 25)
Unseen Data For Predictions: (21, 25)


In [ ]:
# Set up of the refression task with Pycaret (dataset to use and column to predict 'target')
exp_reg101 = setup(data = data, target = 'price', session_id=123)

 
Setup Succesfully Completed.


,Description,Value
0,session_id,123
1,Transform Target,False
2,Transform Target Method,None
3,Original Data,"(184, 25)"
4,Missing Values,False
5,Numeric Features,13
6,Categorical Features,11
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [ ]:
# Above you can see a summary of the dataset and if some operations were made or not such as normalization or the removal of outliers.
# I'm building a first model to try to have the most important features. Then the second model will be built based on the detected features

In [ ]:
# The function compare_models() allows the comparison of several regression models' metrics in a decreasing order of R2 after a cross-validation with 10 folds
compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
0,CatBoost Regressor,1746.2629,6033739.6709,2350.8151,0.8810,0.1541,0.1281,1.5721
1,Random Forest,1693.6359,5984654.2773,2326.4930,0.8795,0.1525,0.1221,0.2406
2,Gradient Boosting Regressor,1726.4246,5984905.2396,2355.6893,0.8746,0.1508,0.1221,0.0748
3,Extreme Gradient Boosting,1788.2019,6480185.9816,2456.0073,0.8628,0.1601,0.1289,0.0661
4,AdaBoost Regressor,2014.9735,6853411.9446,2531.8816,0.8526,0.1770,0.1581,0.0836
5,Extra Trees Regressor,1751.1792,6006718.2142,2394.6747,0.8427,0.1662,0.1340,0.2403
6,Ridge Regression,2007.1864,7419612.2424,2594.5494,0.8272,0.2031,0.1602,0.0075
7,Lasso Regression,2178.7728,9556319.8030,2944.9338,0.8047,0.2218,0.1740,0.0310
8,Light Gradient Boosting Machine,2110.1630,9307386.8922,2906.3376,0.7836,0.1904,0.1553,0.0160
9,Decision Tree,2163.9668,9632221.6598,3031.7389,0.7825,0.1987,0.1543,0.0067


In [ ]:
# The CatBoost regressor is at the top of the list with the highest R2, however I'm choosing the RandomForest because it has the best performance in most metrics
#So let's create it
model = create_model('rf')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,659.9881,660387.7083,812.6424,0.9655,0.0763,0.0650
1,2496.1878,13256813.2208,3640.9907,0.9061,0.1563,0.1338
2,1521.8154,4654085.4773,2157.3330,0.8013,0.1459,0.1185
3,1600.7035,3727740.1703,1930.7357,0.9476,0.1308,0.1145
4,1681.5686,7091830.9387,2663.0492,0.8940,0.1830,0.1141
5,1802.1149,7086453.3114,2662.0393,0.9125,0.1467,0.0995
6,1444.0891,2717636.9257,1648.5257,0.9565,0.1578,0.1348
7,2285.7849,9616043.7541,3100.9746,0.8415,0.2074,0.1626
8,1548.6348,3939194.9298,1984.7405,0.7281,0.1612,0.1388
9,1895.4718,7096356.3369,2663.8987,0.8421,0.1588,0.1397


In [ ]:
#I'll tune the model by trying to find the best hyperparameters and see if it can be improved
tuned = tune_model(model)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,927.9768,1252395.0406,1119.1046,0.9345,0.1030,0.0927
1,3381.1939,30410162.4010,5514.5410,0.7845,0.2020,0.1583
2,1497.7815,3263102.0414,1806.4058,0.8607,0.1429,0.1289
3,1364.2439,4727858.5942,2174.3640,0.9336,0.1100,0.0862
4,1915.9210,9883621.7689,3143.8228,0.8523,0.1700,0.1105
5,2123.7546,8787633.1198,2964.3942,0.8915,0.1405,0.1182
6,1951.3267,5711899.4330,2389.9580,0.9087,0.1963,0.1796
7,2724.7330,12331235.9514,3511.5860,0.7967,0.2267,0.2008
8,1625.8552,4244252.2833,2060.1583,0.7071,0.2024,0.1684
9,1343.1290,2189674.0873,1479.7547,0.9513,0.1265,0.1163


In [ ]:
# The results of the first model are much better than those of the tuned one. I will finalze the first one and then identify which are the most important ones
best = finalize_model(model)

In [ ]:
evaluate_model(best)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [ ]:
# OK, so just like in the first notebook, I can notice that the same most important features were detected
# So now I'm building the new model with only those 5 features instead of the 24 ones in the original dataset

In [ ]:
new_df = dataset[['enginesize', 'compressionratio', 'horsepower',	'highwaympg', 'curbweight', 'price']]
new_df.head()

,enginesize,compressionratio,horsepower,highwaympg,curbweight,price
0,130,9.0,111,27,2548,13495.0
1,130,9.0,111,27,2548,16500.0
2,152,9.0,154,26,2823,16500.0
3,109,10.0,102,30,2337,13950.0
4,136,8.0,115,22,2824,17450.0


In [ ]:
# Again, I'm dividing the dataset into two parts
data = new_df.sample(frac=0.9, random_state=786)
data_unseen = new_df.drop(data.index)

data.reset_index(drop=True, inplace=True)
data_unseen.reset_index(drop=True, inplace=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

Data for Modeling: (184, 6)
Unseen Data For Predictions: (21, 6)


In [ ]:
# Set up
setup(data = data, target = 'price', session_id=123)

 
Setup Succesfully Completed.


,Description,Value
0,session_id,123
1,Transform Target,False
2,Transform Target Method,None
3,Original Data,"(184, 6)"
4,Missing Values,False
5,Numeric Features,5
6,Categorical Features,0
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


(     enginesize  compressionratio  horsepower  highwaympg  curbweight
 0         308.0               8.0       184.0        16.0      3900.0
 1         109.0               8.5        90.0        29.0      2221.0
 2         141.0               9.5       114.0        28.0      2912.0
 3         108.0               9.0        94.0        32.0      2340.0
 4         183.0              21.5       123.0        25.0      3495.0
 ..          ...               ...         ...         ...         ...
 179        97.0              23.0        68.0        38.0      2579.0
 180       183.0              21.5       123.0        25.0      3515.0
 181       108.0               8.8       101.0        29.0      2395.0
 182        92.0               9.0        62.0        37.0      2280.0
 183       122.0               8.7        92.0        32.0      2414.0
 
 [184 rows x 5 columns], 0      40960.0
 1       9980.0
 2      12940.0
 3       9960.0
 4      28176.0
         ...   
 179    13845.0
 180    25

In [ ]:
# Comparison of the models, this time the Extra Trees Regressor is at the first position
compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
0,Extra Trees Regressor,1562.4359,5404037.3566,2207.1369,0.8875,0.1411,0.1129,0.1372
1,Extreme Gradient Boosting,1576.5339,5791698.0953,2273.0665,0.8846,0.1443,0.1122,0.0183
2,CatBoost Regressor,1638.3144,6077754.2518,2311.4217,0.8838,0.1505,0.1189,0.5371
3,Gradient Boosting Regressor,1610.1619,6027318.1605,2336.8284,0.8792,0.1479,0.1146,0.0339
4,Random Forest,1678.2471,6521907.9967,2400.7471,0.8644,0.1560,0.1212,0.2431
5,AdaBoost Regressor,2008.7982,7517574.1736,2636.4373,0.8382,0.1834,0.1591,0.0517
6,Decision Tree,2191.9729,10745039.6667,3191.6066,0.7296,0.2048,0.1576,0.0043
7,Bayesian Ridge,2652.9422,13507973.9337,3552.7258,0.7146,0.3790,0.1946,0.0055
8,Elastic Net,2673.4514,13270076.0234,3528.6842,0.7123,0.2664,0.1992,0.0044
9,Lasso Least Angle Regression,2676.3680,13257333.4810,3526.5650,0.7122,0.2626,0.1998,0.0049


ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=100, n_jobs=-1, oob_score=False,
                    random_state=123, verbose=0, warm_start=False)

In [ ]:
# Creation of the ETR model
etr = create_model('et')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,789.0380,1379484.5834,1174.5146,0.9279,0.0926,0.0678
1,2308.2692,10973968.0162,3312.6980,0.9222,0.1583,0.1313
2,1581.1863,4978268.5109,2231.2034,0.7874,0.1547,0.1218
3,1073.6209,2001557.8668,1414.7642,0.9719,0.0888,0.0778
4,1770.0481,6923649.8530,2631.2829,0.8965,0.1744,0.1182
5,1438.9218,3658074.2432,1912.6093,0.9548,0.1067,0.0909
6,1301.0698,2695170.3238,1641.6974,0.9569,0.1428,0.1129
7,2295.2515,10157740.7391,3187.1211,0.8325,0.2117,0.1692
8,1302.4705,2658740.5050,1630.5645,0.8165,0.1507,0.1302
9,1764.4824,8613718.9245,2934.9138,0.8083,0.1305,0.1089


In [ ]:
# Tuning the model, the first one is better when we compare the yellow lines
tuned_etr = tune_model(etr)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,714.7323,866830.8865,931.0375,0.9547,0.0766,0.0646
1,2524.6166,13278809.8537,3644.0101,0.9059,0.1586,0.1380
2,1547.7774,4520929.1627,2126.2477,0.8070,0.1509,0.1222
3,884.2925,1145648.3142,1070.3496,0.9839,0.0813,0.0662
4,1790.7419,8376549.2907,2894.2269,0.8748,0.1942,0.1104
5,1898.1126,6891885.1005,2625.2400,0.9149,0.1482,0.1183
6,1551.7680,5052011.4650,2247.6680,0.9192,0.1722,0.1356
7,2226.7833,8785012.3109,2963.9521,0.8552,0.2077,0.1696
8,1700.9725,5030878.4877,2242.9620,0.6528,0.1759,0.1570
9,1797.6124,4473767.2438,2115.1282,0.9005,0.1414,0.1355


In [ ]:
# Let's try to bag the model. Results are close to the first ETR model, maybe the tuned one will be better
etr_bagged = ensemble_model(etr)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,680.3002,781064.6637,883.7786,0.9592,0.0726,0.0609
1,2602.8461,15565728.7622,3945.3427,0.8897,0.1646,0.1405
2,1492.9632,4710980.3474,2170.4793,0.7989,0.1479,0.1142
3,855.5295,1111976.9927,1054.5032,0.9844,0.0905,0.0730
4,1734.1132,7048505.5529,2654.9022,0.8947,0.1793,0.1149
5,1795.8059,6106489.6458,2471.1312,0.9246,0.1237,0.1027
6,1217.5481,3003979.7218,1733.1993,0.9520,0.1623,0.1223
7,2370.4865,10850794.2880,3294.0544,0.8211,0.2136,0.1703
8,1759.4827,4972439.6662,2229.8968,0.6568,0.1858,0.1678
9,1455.1887,3170596.1588,1780.6168,0.9294,0.1270,0.1146


In [ ]:
# Bingo! The tuned bagged model is better than the first one:
# tuned_bagged R2: 0.8934 > etr R2: 0.8875
# tuned_bagged RMSE: 2149.9393 < etr RMSE: 2207.1369

tuned_etr_bagged = tune_model(etr_bagged)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,879.1645,1554056.4639,1246.6180,0.9188,0.1035,0.0797
1,2498.2616,13075260.6238,3615.9730,0.9074,0.1692,0.1453
2,1525.5579,4917510.3536,2217.5460,0.7900,0.1497,0.1158
3,1009.2763,1474637.0600,1214.3464,0.9793,0.1035,0.0849
4,1660.7972,6189591.1395,2487.8889,0.9075,0.1722,0.1132
5,1396.0924,3437065.1925,1853.9324,0.9576,0.1012,0.0874
6,1495.3687,3215622.4215,1793.2157,0.9486,0.1595,0.1327
7,2300.5031,9272801.3508,3045.1275,0.8471,0.2118,0.1709
8,1367.3159,2955302.6335,1719.0994,0.7960,0.1624,0.1379
9,1570.9536,5316003.1342,2305.6459,0.8817,0.1176,0.1051


In [ ]:
# I will now compare the predictions of the 3 models on the test set
predict_model(etr)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,1664.2834,6.418937e+06,2533.5621,0.9158,0.1539,0.1206


,enginesize,compressionratio,horsepower,highwaympg,curbweight,price,Label
0,161.0,9.2,156.0,24.0,3151.0,15750.0,16866.8817
1,90.0,9.4,68.0,38.0,1989.0,7609.0,6692.0000
2,98.0,9.4,112.0,29.0,2265.0,9298.0,9313.3500
3,183.0,21.5,123.0,25.0,3495.0,28176.0,25237.6700
4,258.0,8.1,176.0,19.0,4066.0,35550.0,36960.6100
5,98.0,9.0,70.0,34.0,2204.0,8238.0,7954.0400
6,79.0,10.1,60.0,42.0,1837.0,5399.0,6507.2150
7,97.0,23.0,52.0,46.0,2261.0,7775.0,7995.0000
8,98.0,9.0,70.0,34.0,2122.0,8358.0,9047.5500
9,109.0,9.0,85.0,34.0,2209.0,7975.0,8184.3100


In [ ]:
predict_model(etr_bagged)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Bagging Regressor,1583.0582,5.302512e+06,2302.7184,0.9304,0.1532,0.1222


,enginesize,compressionratio,horsepower,highwaympg,curbweight,price,Label
0,161.0,9.2,156.0,24.0,3151.0,15750.0,16933.1220
1,90.0,9.4,68.0,38.0,1989.0,7609.0,6656.4430
2,98.0,9.4,112.0,29.0,2265.0,9298.0,10281.3920
3,183.0,21.5,123.0,25.0,3495.0,28176.0,25499.2140
4,258.0,8.1,176.0,19.0,4066.0,35550.0,36901.6850
5,98.0,9.0,70.0,34.0,2204.0,8238.0,8063.9210
6,79.0,10.1,60.0,42.0,1837.0,5399.0,6498.9043
7,97.0,23.0,52.0,46.0,2261.0,7775.0,8189.4758
8,98.0,9.0,70.0,34.0,2122.0,8358.0,8615.2750
9,109.0,9.0,85.0,34.0,2209.0,7975.0,8058.6570


In [ ]:
predict_model(tuned_etr_bagged)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Bagging Regressor,1532.8797,4.863935e+06,2205.433,0.9362,0.1474,0.1157


,enginesize,compressionratio,horsepower,highwaympg,curbweight,price,Label
0,161.0,9.2,156.0,24.0,3151.0,15750.0,16872.3816
1,90.0,9.4,68.0,38.0,1989.0,7609.0,6628.4058
2,98.0,9.4,112.0,29.0,2265.0,9298.0,9270.2273
3,183.0,21.5,123.0,25.0,3495.0,28176.0,26228.2692
4,258.0,8.1,176.0,19.0,4066.0,35550.0,37132.7376
5,98.0,9.0,70.0,34.0,2204.0,8238.0,8157.1771
6,79.0,10.1,60.0,42.0,1837.0,5399.0,6602.7416
7,97.0,23.0,52.0,46.0,2261.0,7775.0,8027.0298
8,98.0,9.0,70.0,34.0,2122.0,8358.0,8482.1902
9,109.0,9.0,85.0,34.0,2209.0,7975.0,8181.9985


In [ ]:
# Model 	              MAE 	         MSE 	             RMSE 	    R2 	    RMSLE 	MAPE
#tuned_bagged 	       1532.8797 	     4.863935e+06 	   2205.433 	0.9362 	0.1474 	0.1157  ---> best metrics
#etr_bagged 	         1583.0582 	     5.302512e+06 	   2302.7184 	0.9304 	0.1532 	0.1222
#etr 	                 1664.2834 	     6.418937e+06 	   2533.5621 	0.9158 	0.1539 	0.1206

In [ ]:
#Let's finalize the model
final = finalize_model(tuned_etr_bagged)

In [ ]:
evaluate_model(final)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [ ]:
# I will now take a look at how the model predicts data never seen before
preds = predict_model(final, data= data_unseen)
preds

,enginesize,compressionratio,horsepower,highwaympg,curbweight,price,Label
index,,,,,,,
0,136,8.50,110,25,2507,15250.0,14974.9218
1,90,9.60,70,43,1909,6575.0,8577.8896
2,98,7.60,102,30,2128,7957.0,8018.3938
3,92,9.60,58,54,1713,6479.0,6362.1338
4,91,9.00,68,38,1905,6795.0,6337.2760
5,122,8.60,84,32,2425,11245.0,9629.9855
6,122,8.50,88,32,2328,8499.0,7896.7279
7,110,7.50,116,30,2403,9279.0,9427.5938
8,181,9.00,152,25,3060,13499.0,16731.1015


In [ ]:
# The model is not perfect but is giving some satisfying predictions, being sometimes very close to the actual values
# Ex:      Actual     Predicted
           7957.0 	  8018.39
           9279.0 	  9427.59
           6479.0 	  6362.13

In [ ]:
# I will now save the model for later
save_model(final, 'car_price_estimator')

In [52]:
## Let's try to build a model with one more parameter: the brand of the car
dataset['brand']= dataset['CarName'].apply(lambda x :x.split(' ')[0])
dataset.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,enginetype,cylindernumber,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,brand
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0,alfa-romero
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0,alfa-romero
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0,alfa-romero
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0,audi
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0,audi


In [53]:
brand_df = dataset[['brand','enginesize', 'compressionratio', 'horsepower',	'highwaympg', 'curbweight', 'price']]
brand_df.head()


,brand,enginesize,compressionratio,horsepower,highwaympg,curbweight,price
0,alfa-romero,130,9.0,111,27,2548,13495.0
1,alfa-romero,130,9.0,111,27,2548,16500.0
2,alfa-romero,152,9.0,154,26,2823,16500.0
3,audi,109,10.0,102,30,2337,13950.0
4,audi,136,8.0,115,22,2824,17450.0


In [54]:
# See all the brands
brand_df['brand'].unique()

array(['alfa-romero', 'audi', 'bmw', 'chevrolet', 'dodge', 'honda',
       'isuzu', 'jaguar', 'maxda', 'mazda', 'buick', 'mercury',
       'mitsubishi', 'Nissan', 'nissan', 'peugeot', 'plymouth', 'porsche',
       'porcshce', 'renault', 'saab', 'subaru', 'toyota', 'toyouta',
       'vokswagen', 'volkswagen', 'vw', 'volvo'], dtype=object)

In [55]:
# We need to rename some of the brands
correct_names = {'alfa-romero': 'alfa-romeo', 'vokswagen': 'volkswagen','vw': 'volkswagen', 'maxda':'mazda', 'Nissan':'nissan', 'porcshce':'porsche' , 'toyouta': 'toyota'}
brand_df['brand'] = brand_df['brand'].replace(correct_names)

In [56]:
#How many brands do we have?
len(brand_df['brand'].unique())

22

In [57]:
data = brand_df.sample(frac=0.9, random_state=786)
data_unseen = brand_df.drop(data.index)

data.reset_index(drop=True, inplace=True)
data_unseen.reset_index(drop=True, inplace=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

Data for Modeling: (184, 7)
Unseen Data For Predictions: (21, 7)


In [58]:
# Setup
setup(data = data, target = 'price', session_id=16)

 
Setup Succesfully Completed.


,Description,Value
0,session_id,16
1,Transform Target,False
2,Transform Target Method,None
3,Original Data,"(184, 7)"
4,Missing Values,False
5,Numeric Features,5
6,Categorical Features,1
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


(     enginesize  compressionratio  ...  brand_volkswagen  brand_volvo
 0         308.0               8.0  ...               0.0          0.0
 1         109.0               8.5  ...               1.0          0.0
 2         141.0               9.5  ...               0.0          1.0
 3         108.0               9.0  ...               0.0          0.0
 4         183.0              21.5  ...               0.0          0.0
 ..          ...               ...  ...               ...          ...
 179        97.0              23.0  ...               1.0          0.0
 180       183.0              21.5  ...               0.0          0.0
 181       108.0               8.8  ...               0.0          0.0
 182        92.0               9.0  ...               0.0          0.0
 183       122.0               8.7  ...               0.0          0.0
 
 [184 rows x 27 columns], 0      40960.0
 1       9980.0
 2      12940.0
 3       9960.0
 4      28176.0
         ...   
 179    13845.0
 180    2

In [59]:
compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
0,Extra Trees Regressor,1596.1534,5379700.1495,2220.9346,0.8894,0.1334,0.1076,0.1436
1,CatBoost Regressor,1707.2921,6153236.9352,2375.0169,0.8734,0.1422,0.1165,0.6157
2,Extreme Gradient Boosting,1632.3312,5593920.9136,2277.5619,0.8680,0.1388,0.1131,0.0244
3,Gradient Boosting Regressor,1717.9563,6107699.1178,2409.8261,0.8631,0.1414,0.1159,0.0464
4,Random Forest,1648.2864,5738310.0253,2333.1774,0.8604,0.1421,0.1138,0.2446
5,Random Sample Consensus,1846.4134,7648759.0543,2575.9536,0.8578,0.1617,0.1296,0.0812
6,Ridge Regression,2040.8253,8321176.8007,2728.5904,0.8485,0.1891,0.1509,0.0047
7,AdaBoost Regressor,1951.4073,6956099.6682,2559.2153,0.8443,0.1770,0.1527,0.0705
8,Lasso Least Angle Regression,1929.6500,8549582.2453,2724.6838,0.8428,0.1718,0.1351,0.0107
9,Lasso Regression,1942.2868,8731837.0939,2747.1238,0.8403,0.1742,0.1362,0.0062


ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=100, n_jobs=-1, oob_score=False,
                    random_state=16, verbose=0, warm_start=False)

In [60]:
brand_cat = create_model('catboost')
tuned_brand_cat = tune_model(brand_cat)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,1118.6114,2260276.7549,1503.4217,0.8392,0.1347,0.1041
1,1890.9598,5195920.0623,2279.4561,0.9445,0.1312,0.1201
2,1586.9981,6448251.7358,2539.3408,0.8759,0.1468,0.1012
3,1480.3022,4383797.3363,2093.7520,0.7504,0.1592,0.1191
4,1535.1835,3575571.7908,1890.9182,0.9194,0.1399,0.1233
5,1867.1059,6707618.6588,2589.9071,0.9018,0.1398,0.1302
6,1122.1583,2623704.6724,1619.7854,0.9532,0.1183,0.0872
7,3197.0485,19590686.0498,4426.1367,0.8540,0.2108,0.1694
8,2139.2319,7171728.8519,2678.0084,0.8361,0.1705,0.1468
9,1157.0687,2479137.6056,1574.5277,0.9715,0.1023,0.0891


In [61]:
brand_etr = create_model('et')
tuned_brand_etr = tune_model(brand_etr)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,1017.1928,1799305.7475,1341.3820,0.8720,0.1324,0.0992
1,1609.9803,4346420.7701,2084.8071,0.9535,0.1028,0.0904
2,1617.7544,4322055.1513,2078.9553,0.9168,0.1339,0.1206
3,1458.4946,6107488.7596,2471.3334,0.6523,0.1894,0.1144
4,1368.4584,3070047.3835,1752.1551,0.9308,0.1325,0.1118
5,1035.9504,1713472.0495,1308.9966,0.9749,0.0989,0.0876
6,1438.3684,4192446.5417,2047.5465,0.9252,0.1320,0.1046
7,3684.4001,35210821.4855,5933.8707,0.7376,0.2364,0.1684
8,2078.4776,7883873.8351,2807.8237,0.8198,0.1713,0.1373
9,1265.0877,4773623.3414,2184.8623,0.9450,0.1187,0.0956


In [64]:
bagged_brand_etr = ensemble_model(brand_etr)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,1281.6564,2492564.0254,1578.7856,0.8227,0.1309,0.1133
1,1339.5462,2649557.4133,1627.7461,0.9717,0.0969,0.0826
2,1246.3779,3404373.8289,1845.0945,0.9345,0.1268,0.0928
3,1353.5852,5558774.8628,2357.7054,0.6835,0.1761,0.1023
4,1516.1352,4645963.5960,2155.4497,0.8953,0.1335,0.1077
5,1207.1291,2199578.8134,1483.0977,0.9678,0.1105,0.1021
6,1284.9504,2720426.2484,1649.3715,0.9514,0.1198,0.1028
7,3054.7571,21155315.8685,4599.4908,0.8423,0.1803,0.1397
8,2071.9948,7737202.6802,2781.5828,0.8232,0.1732,0.1349
9,1365.8592,3764062.2505,1940.1191,0.9567,0.1041,0.0961


In [65]:
tuned_bagged_brand_etr = tune_model(bagged_brand_etr)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,1162.1267,2282825.5117,1510.9022,0.8376,0.1259,0.1044
1,1314.2092,2292517.9413,1514.1063,0.9755,0.0927,0.0807
2,1271.7778,3606852.8790,1899.1716,0.9306,0.1290,0.0944
3,1370.8838,5863517.4595,2421.4701,0.6661,0.1812,0.1045
4,1466.3025,3883495.3421,1970.6586,0.9125,0.1244,0.1042
5,1285.3516,3260240.6867,1805.6137,0.9523,0.1109,0.1010
6,1232.7085,2673811.8027,1635.1794,0.9523,0.1175,0.0993
7,2882.3962,18506617.8468,4301.9319,0.8621,0.1734,0.1351
8,2051.5467,7095764.7671,2663.7877,0.8378,0.1654,0.1335
9,1416.9256,4175220.8513,2043.3357,0.9519,0.1089,0.1001


In [62]:
brand_rf = create_model('rf')
tuned_brand_rf = tune_model(brand_rf)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,1072.7539,2155380.0449,1468.1213,0.8467,0.1345,0.1050
1,1462.6124,3379367.1746,1838.3055,0.9639,0.0944,0.0876
2,1339.3605,2294343.0560,1514.7089,0.9558,0.1306,0.1156
3,1633.0028,6935801.9856,2633.5911,0.6051,0.1971,0.1269
4,1753.7712,5503242.4224,2345.8991,0.8760,0.1546,0.1300
5,1032.7319,1542100.6814,1241.8135,0.9774,0.1026,0.0912
6,1391.5305,3520684.7918,1876.3488,0.9371,0.1375,0.1099
7,2921.9789,19139083.5821,4374.8238,0.8574,0.1842,0.1405
8,1826.1714,4698559.5981,2167.6161,0.8926,0.1440,0.1350
9,1713.7689,5713637.7097,2390.3217,0.9342,0.1500,0.1339


In [63]:
bagged_brand_rf = ensemble_model(tuned_brand_rf)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,1411.4064,2763792.1704,1662.4657,0.8034,0.1584,0.1397
1,2297.0192,10578781.4020,3252.5039,0.8869,0.1438,0.1228
2,1467.7356,3833067.5747,1957.8222,0.9262,0.1455,0.1208
3,1663.4883,6448217.5852,2539.3341,0.6329,0.1895,0.1358
4,1981.2708,6487660.7712,2547.0887,0.8538,0.1696,0.1482
5,1853.0996,5213787.6670,2283.3720,0.9237,0.1380,0.1344
6,1834.8200,7176245.2339,2678.8515,0.8719,0.1619,0.1342
7,4195.8600,37829869.8946,6150.5991,0.7181,0.2415,0.1872
8,1932.5411,6002624.1386,2450.0253,0.8628,0.1681,0.1497
9,2060.4444,11164481.0260,3341.3292,0.8715,0.1662,0.1416


In [66]:
tuned_bagged_brand_rf = tune_model(bagged_brand_rf)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,1335.6485,2697660.0878,1642.4555,0.8081,0.1500,0.1279
1,1527.7257,3409274.8680,1846.4222,0.9636,0.1022,0.0940
2,1190.5833,2230986.8499,1493.6488,0.9571,0.1301,0.1064
3,1582.2632,6385674.7505,2526.9893,0.6364,0.1895,0.1247
4,1805.4955,5639706.2719,2374.8066,0.8729,0.1547,0.1306
5,1105.8303,1687802.4170,1299.1545,0.9753,0.1050,0.0953
6,1454.7458,3951092.7889,1987.7356,0.9295,0.1393,0.1131
7,3107.7690,22823569.0467,4777.4019,0.8299,0.1964,0.1486
8,2077.6321,6820704.7037,2611.6479,0.8441,0.1593,0.1448
9,1634.1388,5841956.1778,2417.0139,0.9327,0.1357,0.1194


In [68]:
# AMAZING!!! By adding the brands' names, a better model was created but we need to be careful about not having an OVERFITTED model. Let's see the results on the test set for the last model and the tuned random forest
# MODELS KEPT
#    tuned_brand_rf
#    tuned_bagged_brand_etr


predict_model(tuned_brand_rf)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,1617.0502,7211423.14,2685.4093,0.8862,0.1643,0.1325


,enginesize,compressionratio,horsepower,highwaympg,curbweight,brand_alfa-romero,brand_audi,brand_bmw,brand_buick,brand_chevrolet,brand_dodge,brand_honda,brand_isuzu,brand_jaguar,brand_mazda,brand_mercury,brand_mitsubishi,brand_nissan,brand_peugeot,brand_plymouth,brand_porsche,brand_renault,brand_saab,brand_subaru,brand_toyota,brand_volkswagen,brand_volvo,price,Label
0,92.0,9.0,62.0,37.0,2280.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6918.0,7785.1329
1,152.0,21.0,95.0,25.0,3485.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17075.0,17433.8695
2,91.0,9.0,68.0,38.0,1945.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6695.0,7307.9958
3,171.0,9.3,161.0,24.0,2976.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,16558.0,16451.9447
4,92.0,9.0,62.0,38.0,2015.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6488.0,7487.1264
5,110.0,9.0,86.0,33.0,2236.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7895.0,8883.5199
6,90.0,9.4,68.0,38.0,1989.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6692.0,6739.5707
7,92.0,9.2,76.0,34.0,2024.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7295.0,7485.4366
8,122.0,22.7,64.0,42.0,2443.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10795.0,10408.0362
9,92.0,9.0,62.0,32.0,3110.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8778.0,10689.5194


In [69]:
predict_model(tuned_bagged_brand_etr)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Bagging Regressor,1450.6376,5.853656e+06,2419.4329,0.9076,0.1437,0.112


,enginesize,compressionratio,horsepower,highwaympg,curbweight,brand_alfa-romero,brand_audi,brand_bmw,brand_buick,brand_chevrolet,brand_dodge,brand_honda,brand_isuzu,brand_jaguar,brand_mazda,brand_mercury,brand_mitsubishi,brand_nissan,brand_peugeot,brand_plymouth,brand_porsche,brand_renault,brand_saab,brand_subaru,brand_toyota,brand_volkswagen,brand_volvo,price,Label
0,92.0,9.0,62.0,37.0,2280.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6918.0,8070.5482
1,152.0,21.0,95.0,25.0,3485.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17075.0,16187.3981
2,91.0,9.0,68.0,38.0,1945.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6695.0,6993.5044
3,171.0,9.3,161.0,24.0,2976.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,16558.0,16899.8280
4,92.0,9.0,62.0,38.0,2015.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6488.0,7415.0105
5,110.0,9.0,86.0,33.0,2236.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7895.0,8676.3909
6,90.0,9.4,68.0,38.0,1989.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6692.0,7229.4624
7,92.0,9.2,76.0,34.0,2024.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7295.0,7270.6507
8,122.0,22.7,64.0,42.0,2443.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10795.0,10609.4170
9,92.0,9.0,62.0,32.0,3110.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8778.0,12460.9748


In [70]:
brand_final_rf = finalize_model(tuned_brand_rf)

In [71]:
brand_final_etr = finalize_model(tuned_bagged_brand_etr)

In [72]:
# I will now take a look at how the model predicts data never seen before. I'm creating a difference column to see how close the model was to the actuel data
preds = predict_model(brand_final_rf, data= data_unseen)
preds['diff']= preds['price']- preds['Label']
preds 

,brand,enginesize,compressionratio,horsepower,highwaympg,curbweight,price,Label,diff
index,,,,,,,,,
0,audi,136,8.50,110,25,2507,15250.0,14888.7039,361.2961
1,chevrolet,90,9.60,70,43,1909,6575.0,6412.3563,162.6437
2,dodge,98,7.60,102,30,2128,7957.0,8956.9771,-999.9771
3,honda,92,9.60,58,54,1713,6479.0,6868.0976,-389.0976
4,mazda,91,9.00,68,38,1905,6795.0,6760.5963,34.4037
5,mazda,122,8.60,84,32,2425,11245.0,9916.6524,1328.3476
6,mitsubishi,122,8.50,88,32,2328,8499.0,8545.4771,-46.4771
7,mitsubishi,110,7.50,116,30,2403,9279.0,9877.7483,-598.7483
8,nissan,181,9.00,152,25,3060,13499.0,17559.1811,-4060.1811


In [73]:
# Let's see the root mean squared error for the 1st model

from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(preds['price'], preds['Label']))
rms 

2175.8347110045543

In [74]:
preds2 = predict_model(brand_final_etr, data= data_unseen)
preds2['diff']= preds2['price']- preds2['Label']
preds2

,brand,enginesize,compressionratio,horsepower,highwaympg,curbweight,price,Label,diff
index,,,,,,,,,
0,audi,136,8.50,110,25,2507,15250.0,14837.0810,412.9190
1,chevrolet,90,9.60,70,43,1909,6575.0,6139.2734,435.7266
2,dodge,98,7.60,102,30,2128,7957.0,8357.5614,-400.5614
3,honda,92,9.60,58,54,1713,6479.0,5832.2887,646.7113
4,mazda,91,9.00,68,38,1905,6795.0,6220.8867,574.1133
5,mazda,122,8.60,84,32,2425,11245.0,9603.4091,1641.5909
6,mitsubishi,122,8.50,88,32,2328,8499.0,7803.8761,695.1239
7,mitsubishi,110,7.50,116,30,2403,9279.0,9525.2688,-246.2688
8,nissan,181,9.00,152,25,3060,13499.0,16025.4959,-2526.4959


In [75]:
# Let's see the RMSE of the 2nd model
rms = sqrt(mean_squared_error(preds2['price'], preds2['Label']))
rms 

2219.5934554287796

In [ ]:
# The 2 models are very close. The fact that the dataset didn't lots of values has to be taken into consideration also for the training part
# I'll choose the first model (tuned  random forest) for the web application

In [76]:
save_model(brand_final_rf, 'car_rf_model')

Transformation Pipeline and Model Succesfully Saved
